In [1]:
# IMPORT NECESSARY LIBRARIES AND DATA
import pandas as pd
import numpy as np
import bamboolib as bam #easy to do data manipulation and highly recommend going to https://bamboolib.8080labs.com/ and that will help you follow along

In [2]:
data_url = 'https://data.boston.gov/dataset/6220d948-eae2-4e4b-8723-2dc8e67722a3/resource/b973d8cb-eeb2-4e7e-99da-c92938efc9c0/download/tmpomwevvy8.csv'
crime = pd.read_csv(data_url, header=0)


In [11]:
crime

       INCIDENT_NUMBER  OFFENSE_CODE                    OFFENSE_DESCRIPTION  \
0            232010316          1106        FRAUD - CREDIT CARD / ATM FRAUD   
1            232014572          2670        HARASSMENT/ CRIMINAL HARASSMENT   
2            232016999          3207                       PROPERTY - FOUND   
3            232017000          3207                       PROPERTY - FOUND   
4            232011980          1109                           FRAUD - WIRE   
...                ...           ...                                    ...   
14522        232009411          3115                     INVESTIGATE PERSON   
14523        232009427          3006          SICK/INJURED/MEDICAL - PERSON   
14524        232009415          1832     SICK ASSIST - DRUG RELATED ILLNESS   
14525        232009421          3831  M/V - LEAVING SCENE - PROPERTY DAMAGE   
14526        232009424          3115                     INVESTIGATE PERSON   

      DISTRICT REPORTING_AREA  SHOOTING        OCCU

In [3]:
crime.columns

Index(['INCIDENT_NUMBER', 'OFFENSE_CODE', 'OFFENSE_CODE_GROUP',
       'OFFENSE_DESCRIPTION', 'DISTRICT', 'REPORTING_AREA', 'SHOOTING',
       'OCCURRED_ON_DATE', 'YEAR', 'MONTH', 'DAY_OF_WEEK', 'HOUR', 'UCR_PART',
       'STREET', 'Lat', 'Long', 'Location'],
      dtype='object')

In [5]:
crime = crime.drop(columns=['OFFENSE_CODE_GROUP','UCR_PART'])

KeyError: "['OFFENSE_CODE_GROUP', 'UCR_PART'] not found in axis"

In [6]:
crime

       INCIDENT_NUMBER  OFFENSE_CODE                    OFFENSE_DESCRIPTION  \
0            232010316          1106        FRAUD - CREDIT CARD / ATM FRAUD   
1            232014572          2670        HARASSMENT/ CRIMINAL HARASSMENT   
2            232016999          3207                       PROPERTY - FOUND   
3            232017000          3207                       PROPERTY - FOUND   
4            232011980          1109                           FRAUD - WIRE   
...                ...           ...                                    ...   
14522        232009411          3115                     INVESTIGATE PERSON   
14523        232009427          3006          SICK/INJURED/MEDICAL - PERSON   
14524        232009415          1832     SICK ASSIST - DRUG RELATED ILLNESS   
14525        232009421          3831  M/V - LEAVING SCENE - PROPERTY DAMAGE   
14526        232009424          3115                     INVESTIGATE PERSON   

      DISTRICT REPORTING_AREA  SHOOTING        OCCU

In [28]:
mode = crime['DISTRICT'].mode()[0]
crime['DISTRICT'] = crime['DISTRICT'].fillna(mode)

In [31]:
district_means = crime.groupby('DISTRICT')[['Lat', 'Long']].mean()
crime[['Lat', 'Long']] = crime.groupby('DISTRICT')[['Lat', 'Long']].transform(lambda x: x.fillna(x.mean()))


In [32]:
crime

       INCIDENT_NUMBER  OFFENSE_CODE                    OFFENSE_DESCRIPTION  \
0            232010316          1106        FRAUD - CREDIT CARD / ATM FRAUD   
1            232014572          2670        HARASSMENT/ CRIMINAL HARASSMENT   
2            232016999          3207                       PROPERTY - FOUND   
3            232017000          3207                       PROPERTY - FOUND   
4            232011980          1109                           FRAUD - WIRE   
...                ...           ...                                    ...   
14522        232009411          3115                     INVESTIGATE PERSON   
14523        232009427          3006          SICK/INJURED/MEDICAL - PERSON   
14524        232009415          1832     SICK ASSIST - DRUG RELATED ILLNESS   
14525        232009421          3831  M/V - LEAVING SCENE - PROPERTY DAMAGE   
14526        232009424          3115                     INVESTIGATE PERSON   

       DISTRICT REPORTING_AREA  SHOOTING        OCC

In [ ]:
from geopy.geocoders import Nominatim

# load crime data

# create geolocator object
geolocator = Nominatim(user_agent='crime_app')

# define function to get location from latitude and longitude values
def get_location(lat, long):
    try:
        location = geolocator.reverse(f"{lat}, {long}")
        return location.address.split(',')[-3].strip()
    except:
        return None

# create lookup table for latitude and longitude values
lat_long_df = crime[['Lat', 'Long']].drop_duplicates()
lat_long_df['Location'] = lat_long_df.apply(lambda row: get_location(row['Lat'], row['Long']), axis=1)

# merge lookup table with crime data to impute missing location values
crime = pd.merge(crime, lat_long_df[['Lat', 'Long', 'Location']], on=['Lat', 'Long'], how='left')
crime['Location'] = crime['Location_x'].fillna(crime['Location_y'])
crime.drop(['Location_x', 'Location_y'], axis=1, inplace=True)
